In [ ]:
!pip install agno
!pip install duckduckgo-search fastapi uvicorn openai

In [ ]:
!pip install pypdf
!pip install lancedb
!pip install tantivy

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.embedder.openai import OpenAIEmbedder
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType
import os
from dotenv import load_dotenv
from google.colab import userdata
os.environ["OPENAI_API_KEY"]= userdata.get("OP_SK")

In [ ]:
pip install pylance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.5/36.5 MB 44.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Food Security RAG** **AGENT**

In [ ]:
from agno.models.openai import OpenAIChat
food_agent = Agent(
    name="food_agent",
    role="Search your knowledge base for  Food Security queries",
    model= OpenAIChat(id="gpt-4o"),
    description="handles Food Security queries",
    instructions=[
        "Search your knowledge base for  Food Security queries",
        "If the question is better suited for the web, search the web to fill in gaps.",
        "Prefer the information in your knowledge base over the web results."
    ],
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://drive.google.com/file/d/1__65iWzkZ8qrsTuPMl_V-ntoGLkk5ziy/view?usp=sharing"],  # 👈 Replace with the local path to your PDF
        vector_db=LanceDb(
            uri="tmp/lancedb",
            table_name="food_security",
            search_type=SearchType.hybrid,
            embedder=OpenAIEmbedder(id="text-embedding-3-small"),
        ),
    ),
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

**Clinical Data RAG AGENT**

In [ ]:
clinical_agent = Agent(
    name="clinical_agent",
    role="Search your knowledge base for  the matching clinical data",
    model=OpenAIChat(id="gpt-4o"),
    description="handles quires whihc requires clinical data",
    instructions=[
        "Search your knowledge base for  the matching clinical data"
    ],
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://drive.google.com/file/d/17XZvrdNL2cc4WAuE9zojPNrI0VtOPUGU/view?usp=drive_link"],  # 👈 Replace with the local path to your PDF
        vector_db=LanceDb(
            uri="tmp/lancedb",
            table_name="Clinical_data",
            search_type=SearchType.hybrid,
            embedder=OpenAIEmbedder(id="text-embedding-3-small"),
        ),
    ),
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)

**WEB UI search AGENT**

In [ ]:
from agno.tools.duckduckgo import DuckDuckGoTools

web_search_agent = Agent(
    name="web_search_agent",
    role="Search the web for information",
    model= OpenAIChat(id="gpt-4o"),
    description="Search the web for the information",
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)

**Combining all the agents as a Team**

In [ ]:
from agno.team import Team

agent_team = Team(
    mode="coordinate",
    members=[web_search_agent,clinical_agent, food_agent],
    model= OpenAIChat(id="gpt-4o"),
    instructions=["use the relavent agent based on user query and  at the end specifically mention which agents you used to get the results"],
    show_tool_calls=True,
    markdown=True,
)


In [ ]:
agent_team.print_response("SIX TRANSFORMATION PATHWAYS TO ADDRESS THE MAJOR DRIVERS OF FOOD INSECURITY AND MALNUTRITION", stream=True)


Output()

ERROR    Error searching for documents: The lance library is required to use this function. Please install with    
         `pip install pylance`.

In [ ]:
agent_team.print_response("can you get info regarding this clinical testARdioprotection in Myocardial Infarction", stream=True)

Output()

In [ ]:
agent_team.print_response("Retrieve Average Price: Milk, Fresh, Whole, Fortified (Cost per Gallon/3.8Liters) in U.S. City Average (APU0000709112) from 1992 to current date on Monthly Frequency with x-axis containing date and y-axis represents price of gas.", stream=True)

Output()